In [ ]:
%load_ext cudf

The cudf module is not an IPython extension.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

# **21_IVT**

In [ ]:
df_21_IVT = pd.read_csv('data/STData/21/21_IVT.csv')

In [ ]:
df_21_IVT.head()

In [ ]:
df_21_IVT.columns

In [ ]:
df_21_IVT.shape

In [ ]:
df_21_IVT.info()

In [ ]:
df_21_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_21_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_21_IVT['QuestionKey'].unique()

In [ ]:
df_21_IVT['Timestamp'] = pd.to_datetime(df_21_IVT['Timestamp'])

In [ ]:
df_21_IVT.head(3)

In [ ]:
df_21_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_21_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_21_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_21_IVT.isnull().sum()

In [ ]:
df_21_IVT.head()

In [ ]:
df_21_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_21_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_21_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_21_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_21_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_21_IVT.describe()

In [ ]:
df_21_IVT.head(3)

In [ ]:
df_21_IVT['Timestamp'] = pd.to_datetime(df_21_IVT['Timestamp'])

In [ ]:
df_21_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_21_IVT['Timestamp'], y=df_21_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_21_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_21_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_21_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_21_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_21_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_21_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_21_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_21_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_21_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_21_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_21_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_21_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_21_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_21_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_21_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_21_IVT.columns

In [ ]:
fix_1_df = df_21_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_21_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **22_IVT**

In [ ]:
df_22_IVT = pd.read_csv('data/STData/22/22_IVT.csv')

In [ ]:
df_22_IVT.head()

In [ ]:
df_22_IVT.columns

In [ ]:
df_22_IVT.shape

In [ ]:
df_22_IVT.info()

In [ ]:
df_22_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_22_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_22_IVT['QuestionKey'].unique()

In [ ]:
df_22_IVT['Timestamp'] = pd.to_datetime(df_22_IVT['Timestamp'])

In [ ]:
df_22_IVT.head(3)

In [ ]:
df_22_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_22_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_22_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_22_IVT.isnull().sum()

In [ ]:
df_22_IVT.head()

In [ ]:
df_22_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_22_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_22_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_22_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_22_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_22_IVT.describe()

In [ ]:
df_22_IVT.head(3)

In [ ]:
df_22_IVT['Timestamp'] = pd.to_datetime(df_22_IVT['Timestamp'])

In [ ]:
df_22_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_22_IVT['Timestamp'], y=df_22_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_22_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_22_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_22_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_22_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_22_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_22_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_22_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_22_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_22_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_22_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_22_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_22_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_22_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_22_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_22_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_22_IVT.columns

In [ ]:
fix_1_df = df_22_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_22_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **23_IVT**

In [ ]:
df_23_IVT = pd.read_csv('data/STData/23/23_IVT.csv')

In [ ]:
df_23_IVT.head()

In [ ]:
df_23_IVT.columns

In [ ]:
df_23_IVT.shape

In [ ]:
df_23_IVT.info()

In [ ]:
df_23_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_23_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_23_IVT['QuestionKey'].unique()

In [ ]:
df_23_IVT['Timestamp'] = pd.to_datetime(df_23_IVT['Timestamp'])

In [ ]:
df_23_IVT.head(3)

In [ ]:
df_23_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_23_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_23_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_23_IVT.isnull().sum()

In [ ]:
df_23_IVT.head()

In [ ]:
df_23_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_23_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_23_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_23_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_23_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_23_IVT.describe()

In [ ]:
df_23_IVT.head(3)

In [ ]:
df_23_IVT['Timestamp'] = pd.to_datetime(df_23_IVT['Timestamp'])

In [ ]:
df_23_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_23_IVT['Timestamp'], y=df_23_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_23_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_23_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_23_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_23_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_23_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_23_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_23_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_23_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_23_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_23_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_23_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_23_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_23_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_23_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_23_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_23_IVT.columns

In [ ]:
fix_1_df = df_23_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_23_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **24_IVT**

In [ ]:
df_24_IVT = pd.read_csv('data/STData/24/24_IVT.csv')

In [ ]:
df_24_IVT.head()

In [ ]:
df_24_IVT.columns

In [ ]:
df_24_IVT.shape

In [ ]:
df_24_IVT.info()

In [ ]:
df_24_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_24_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_24_IVT['QuestionKey'].unique()

In [ ]:
df_24_IVT['Timestamp'] = pd.to_datetime(df_24_IVT['Timestamp'])

In [ ]:
df_24_IVT.head(3)

In [ ]:
df_24_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_24_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_24_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_24_IVT.isnull().sum()

In [ ]:
df_24_IVT.head()

In [ ]:
df_24_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_24_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_24_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_24_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_24_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_24_IVT.describe()

In [ ]:
df_24_IVT.head(3)

In [ ]:
df_24_IVT['Timestamp'] = pd.to_datetime(df_24_IVT['Timestamp'])

In [ ]:
df_24_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_24_IVT['Timestamp'], y=df_24_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_24_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_24_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_24_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_24_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_24_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_24_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_24_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_24_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_24_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_24_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_24_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_24_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_24_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_24_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_24_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_24_IVT.columns

In [ ]:
fix_1_df = df_24_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_24_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **25_IVT**

In [ ]:
df_25_IVT = pd.read_csv('data/STData/25/25_IVT.csv')

In [ ]:
df_25_IVT.head()

In [ ]:
df_25_IVT.columns

In [ ]:
df_25_IVT.shape

In [ ]:
df_25_IVT.info()

In [ ]:
df_25_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_25_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_25_IVT['QuestionKey'].unique()

In [ ]:
df_25_IVT['Timestamp'] = pd.to_datetime(df_25_IVT['Timestamp'])

In [ ]:
df_25_IVT.head(3)

In [ ]:
df_25_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_25_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_25_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_25_IVT.isnull().sum()

In [ ]:
df_25_IVT.head()

In [ ]:
df_25_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_25_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_25_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_25_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_25_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_25_IVT.describe()

In [ ]:
df_25_IVT.head(3)

In [ ]:
df_25_IVT['Timestamp'] = pd.to_datetime(df_25_IVT['Timestamp'])

In [ ]:
df_25_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_25_IVT['Timestamp'], y=df_25_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_25_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_25_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_25_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_25_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_25_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_25_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_25_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_25_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_25_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_25_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_25_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_25_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_25_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_25_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_25_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_25_IVT.columns

In [ ]:
fix_1_df = df_25_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_25_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **26_IVT**

In [ ]:
df_26_IVT = pd.read_csv('data/STData/26/26_IVT.csv')

In [ ]:
df_26_IVT.head()

In [ ]:
df_26_IVT.columns

In [ ]:
df_26_IVT.shape

In [ ]:
df_26_IVT.info()

In [ ]:
df_26_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_26_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_26_IVT['QuestionKey'].unique()

In [ ]:
df_26_IVT['Timestamp'] = pd.to_datetime(df_26_IVT['Timestamp'])

In [ ]:
df_26_IVT.head(3)

In [ ]:
df_26_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_26_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_26_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_26_IVT.isnull().sum()

In [ ]:
df_26_IVT.head()

In [ ]:
df_26_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_26_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_26_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_26_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_26_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_26_IVT.describe()

In [ ]:
df_26_IVT.head(3)

In [ ]:
df_26_IVT['Timestamp'] = pd.to_datetime(df_26_IVT['Timestamp'])

In [ ]:
df_26_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_26_IVT['Timestamp'], y=df_26_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_26_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_26_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_26_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_26_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_26_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_26_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_26_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_26_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_26_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_26_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_26_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_26_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_26_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_26_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_26_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_26_IVT.columns

In [ ]:
fix_1_df = df_26_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_26_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **27_IVT**

In [ ]:
df_27_IVT = pd.read_csv('data/STData/27/27_IVT.csv')

In [ ]:
df_27_IVT.head()

In [ ]:
df_27_IVT.columns

In [ ]:
df_27_IVT.shape

In [ ]:
df_27_IVT.info()

In [ ]:
df_27_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_27_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_27_IVT['QuestionKey'].unique()

In [ ]:
df_27_IVT['Timestamp'] = pd.to_datetime(df_27_IVT['Timestamp'])

In [ ]:
df_27_IVT.head(3)

In [ ]:
df_27_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_27_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_27_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_27_IVT.isnull().sum()

In [ ]:
df_27_IVT.head()

In [ ]:
df_27_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_27_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_27_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_27_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_27_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_27_IVT.describe()

In [ ]:
df_27_IVT.head(3)

In [ ]:
df_27_IVT['Timestamp'] = pd.to_datetime(df_27_IVT['Timestamp'])

In [ ]:
df_27_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_27_IVT['Timestamp'], y=df_27_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_27_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_27_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_27_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_27_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_27_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_27_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_27_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_27_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_27_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_27_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_27_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_27_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_27_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_27_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_27_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_27_IVT.columns

In [ ]:
fix_1_df = df_27_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_27_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **28_IVT**

In [ ]:
df_28_IVT = pd.read_csv('data/STData/28/28_IVT.csv')

In [ ]:
df_28_IVT.head()

In [ ]:
df_28_IVT.columns

In [ ]:
df_28_IVT.shape

In [ ]:
df_28_IVT.info()

In [ ]:
df_28_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_28_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_28_IVT['QuestionKey'].unique()

In [ ]:
df_28_IVT['Timestamp'] = pd.to_datetime(df_28_IVT['Timestamp'])

In [ ]:
df_28_IVT.head(3)

In [ ]:
df_28_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_28_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_28_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_28_IVT.isnull().sum()

In [ ]:
df_28_IVT.head()

In [ ]:
df_28_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_28_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_28_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_28_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_28_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_28_IVT.describe()

In [ ]:
df_28_IVT.head(3)

In [ ]:
df_28_IVT['Timestamp'] = pd.to_datetime(df_28_IVT['Timestamp'])

In [ ]:
df_28_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_28_IVT['Timestamp'], y=df_28_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_28_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_28_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_28_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_28_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_28_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_28_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_28_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_28_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_28_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_28_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_28_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_28_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_28_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_28_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_28_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_28_IVT.columns

In [ ]:
fix_1_df = df_28_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_28_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **29_IVT**

In [ ]:
df_29_IVT = pd.read_csv('data/STData/29/29_IVT.csv')

In [ ]:
df_29_IVT.head()

In [ ]:
df_29_IVT.columns

In [ ]:
df_29_IVT.shape

In [ ]:
df_29_IVT.info()

In [ ]:
df_29_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_29_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_29_IVT['QuestionKey'].unique()

In [ ]:
df_29_IVT['Timestamp'] = pd.to_datetime(df_29_IVT['Timestamp'])

In [ ]:
df_29_IVT.head(3)

In [ ]:
df_29_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_29_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_29_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_29_IVT.isnull().sum()

In [ ]:
df_29_IVT.head()

In [ ]:
df_29_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_29_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_29_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_29_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_29_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_29_IVT.describe()

In [ ]:
df_29_IVT.head(3)

In [ ]:
df_29_IVT['Timestamp'] = pd.to_datetime(df_29_IVT['Timestamp'])

In [ ]:
df_29_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_29_IVT['Timestamp'], y=df_29_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_29_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_29_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_29_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_29_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_29_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_29_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_29_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_29_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_29_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_29_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_29_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_29_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_29_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_29_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_29_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_29_IVT.columns

In [ ]:
fix_1_df = df_29_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_29_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.

# **30_IVT**

In [ ]:
df_30_IVT = pd.read_csv('data/STData/30/30_IVT.csv')

In [ ]:
df_30_IVT.head()

In [ ]:
df_30_IVT.columns

In [ ]:
df_30_IVT.shape

In [ ]:
df_30_IVT.info()

In [ ]:
df_30_IVT.isnull().sum()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_30_IVT.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

In [ ]:
df_30_IVT['QuestionKey'].unique()

In [ ]:
df_30_IVT['Timestamp'] = pd.to_datetime(df_30_IVT['Timestamp'])

In [ ]:
df_30_IVT.head(3)

In [ ]:
df_30_IVT['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_30_IVT['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_30_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_30_IVT.isnull().sum()

In [ ]:
df_30_IVT.head()

In [ ]:
df_30_IVT['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_30_IVT['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_30_IVT.drop('Row', axis=1, inplace=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

sns.scatterplot(data=df_30_IVT, x='Gaze X', y='Gaze Y', ax=axes[0])
axes[0].set_title('Gaze X vs Gaze Y')

sns.scatterplot(data=df_30_IVT, x='Interpolated Gaze X', y='Interpolated Gaze Y', ax=axes[1])
axes[1].set_title('Interpolated Gaze X vs Interpolated Gaze Y')

plt.tight_layout()
plt.show()

# Gaze and Interpolated Gaze Scatter Plots

The scatter plots above visualize the relationship between the x and y coordinates of both the raw gaze data and the interpolated gaze data.

- **Gaze X vs Gaze Y:** This plot shows the raw gaze coordinates. The scattered points indicate the locations on the screen where the participant was looking. The density of points in certain areas might suggest regions of interest.
- **Interpolated Gaze X vs Interpolated Gaze Y:** This plot shows the interpolated gaze coordinates. Interpolation is often used to fill in gaps in the raw gaze data, providing a smoother representation of the gaze path. Comparing this plot to the raw gaze plot can show the effect of the interpolation process.

Both plots can help in understanding the distribution of gaze points across the screen and identifying potential patterns or biases in eye movements.

In [ ]:
df_30_IVT.describe()

In [ ]:
df_30_IVT.head(3)

In [ ]:
df_30_IVT['Timestamp'] = pd.to_datetime(df_30_IVT['Timestamp'])

In [ ]:
df_30_IVT.columns

In [ ]:
cols = ['Gaze X', 'Gaze Y',
       'Interpolated Gaze X', 'Interpolated Gaze Y', 'Interpolated Distance',
       'Gaze Velocity', 'Gaze Acceleration', 'Fixation Index',
       'Fixation Index by Stimulus', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion', 'Saccade Index', 'Saccade Index by Stimulus',
       'Saccade Start', 'Saccade End', 'Saccade Duration', 'Saccade Amplitude',
       'Saccade Peak Velocity', 'Saccade Peak Acceleration',
       'Saccade Peak Deceleration', 'Saccade Direction']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    sns.lineplot(x=df_30_IVT['Timestamp'], y=df_30_IVT[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
df_30_IVT.head()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df_30_IVT[['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus']].isnull(), cmap='viridis')
plt.show()

# Observation

The `Fixation Index`, `Fixation Index by Stimulus`, `Saccade Index` and `Saccade Index by Stimulus` columns are essentially just sequence numbers for identified events. While they indicate the order of fixations and saccades, they don't provide meaningful features for a machine learning model attempting to predict or classify eye movement patterns. Therefore, we will drop these columns as they are not useful for model building.

In [ ]:
df_30_IVT.drop(['Fixation Index', 'Fixation Index by Stimulus', 'Saccade Index', 'Saccade Index by Stimulus'], axis=1, inplace=True)

In [ ]:
plt.figure(figsize=(14,10))
sns.scatterplot(data=df_30_IVT, x='Fixation X', y='Fixation Y')
plt.title('Fixation X vs Fixation Y')
plt.show()

In [ ]:
df_30_IVT['Fixation Start'].describe()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_30_IVT['Fixation Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Fixation Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Fixation Start')

sns.histplot(df_30_IVT['Fixation End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Fixation End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Fixation End')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

sns.histplot(df_30_IVT['Saccade Start'], bins=100, kde=True, ax=axes[0])
axes[0].set_xlabel('Saccade Start')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Saccade Start')

sns.histplot(df_30_IVT['Saccade End'], bins=100, kde=True, ax=axes[1])
axes[1].set_xlabel('Saccade End')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Saccade End')

plt.tight_layout()
plt.show()

# Observation on Fixation and Saccade Timestamps

Upon examining the time series plots of 'Fixation Start', 'Fixation End', `Saccade Start`, and `Saccade End` against the `Timestamp`, we observe a clear linear, diagonal pattern. This indicates that these values are largely sequential and directly related to the progress of time in the data recording.

Furthermore, the histograms of these features show distributions that, while informative about the timing of events, don't necessarily reveal complex patterns that would be highly predictive for a machine learning model.

Crucially, the dataset already contains `Fixation Duration` and `Saccade Duration` columns. These duration features capture the length of each event, which is often a more directly relevant metric for understanding eye movement behavior than the absolute start and end times. Since the duration can be derived from the start and end times (Duration = End - Start), the start and end time columns introduce redundancy and do not provide substantial additional, independent information for modeling purposes.

Therefore, to simplify the dataset and focus on the most informative features for potential machine movement analysis or modeling, we will drop the `Fixation Start`, `Fixation End`, `Saccade Start`, and `Saccade End` columns.

In [ ]:
df_30_IVT.drop(['Fixation Start', 'Fixation End', 'Saccade Start', 'Saccade End'], axis=1, inplace=True)

In [ ]:
df_30_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_30_IVT[['Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y']].isnull(), cmap='viridis')
plt.show()


# Observations on Gaze and Interpolated Gaze Data

Based on the scatter plots of 'Gaze X' vs 'Gaze Y' and 'Interpolated Gaze X' vs 'Interpolated Gaze Y', we observe that the distributions of the raw and interpolated gaze points appear very similar. The spatial patterns of where the participant was looking are consistent between the two sets of coordinates.

Furthermore, the heatmap of null values for these columns ('Gaze X', 'Gaze Y', 'Interpolated Gaze X', 'Interpolated Gaze Y') reveals that the missing values are present in the same rows for both the raw and interpolated gaze coordinates. This suggests that the interpolation process did not fill in the gaps in the raw gaze data for these specific instances.

Given that the interpolated gaze data shows the same spatial distribution and the same pattern of null values as the raw gaze data, it appears that the interpolation did not significantly alter or complete the data in this case. Therefore, keeping both the raw and interpolated gaze columns might be redundant, and one set could potentially be dropped to simplify the dataset without losing significant information.

In [ ]:
df_30_IVT.drop(['Interpolated Gaze X', 'Interpolated Gaze Y'], axis=1, inplace=True)

In [ ]:
df_30_IVT.head()

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df_30_IVT.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_30_IVT.columns

In [ ]:
fix_1_df = df_30_IVT.dropna(subset=['Fixation Duration'])
sac_1_df = df_30_IVT.dropna(subset=['Saccade Duration'])

In [ ]:
fix_1_df.shape

In [ ]:
sac_1_df.shape

In [ ]:
fix_1_feature = fix_1_df.groupby('QuestionKey').agg({
    'Fixation Duration': ['count','mean','max','sum','var'],
    'Fixation Dispersion': ['mean','max'],
    'Fixation X': ['var'],   # screen spread X
    'Fixation Y': ['var']    # screen spread Y
})

In [ ]:
fix_1_feature.columns = ['fix_count','fix_mean_dur','fix_max_dur','fix_total_time',
                        'fix_dur_var','fix_disp_mean','fix_disp_max',
                        'fix_x_var','fix_y_var']

In [ ]:
fix_1_feature

In [ ]:
sac_1_features = sac_1_df.groupby('QuestionKey').agg({
    'Saccade Duration': ['count','mean','sum'],
    'Saccade Amplitude': ['mean','max'],
    'Saccade Peak Velocity': ['mean','max'],
    'Saccade Peak Acceleration': ['mean'],
    'Saccade Peak Deceleration': ['mean'],
    'Saccade Direction': ['var']   # direction variance
})

In [ ]:
sac_1_features.columns = ['sac_count','sac_mean_dur','sac_total_time',
                        'sac_amp_mean','sac_amp_max',
                        'sac_vel_mean','sac_vel_max',
                        'sac_acc_mean','sac_dec_mean','sac_dir_var']

In [ ]:
sac_1_features

In [ ]:
ivt_1_features = fix_1_feature.join(sac_1_features, how='outer').fillna(0)

In [ ]:
ivt_1_features

In [ ]:
ivt_1_features['fix_sac_count_ratio'] = ivt_1_features['fix_count'] / (ivt_1_features['sac_count']+1e-5)
ivt_1_features['fix_sac_time_ratio']  = ivt_1_features['fix_total_time'] / (ivt_1_features['sac_total_time']+1e-5)

In [ ]:
ivt_1_features

# Aggregation of Fixation and Saccade Features

In the preceding code cells, we performed aggregation on the `fix_1_df` and `sac_1_df` DataFrames, which contain the cleaned fixation and saccade data, respectively. The goal of this aggregation was to create a summary of eye-tracking metrics for each `QuestionKey`.

For fixations, we calculated:
- Count of fixations (`fix_count`)
- Mean, max, sum, and variance of fixation duration (`fix_mean_dur`, `fix_max_dur`, `fix_total_time`, `fix_dur_var`)
- Mean and max of fixation dispersion (`fix_disp_mean`, `fix_disp_max`)
- Variance of fixation X and Y coordinates (`fix_x_var`, `fix_y_var`) to represent screen spread.

For saccades, we calculated:
- Count of saccades (`sac_count`)
- Mean and sum of saccade duration (`sac_mean_dur`, `sac_total_time`)
- Mean and max of saccade amplitude (`sac_amp_mean`, `sac_amp_max`)
- Mean and max of saccade peak velocity (`sac_vel_mean`, `sac_vel_max`)
- Mean of saccade peak acceleration and deceleration (`sac_acc_mean`, `sac_dec_mean`)
- Variance of saccade direction (`sac_dir_var`).

Finally, we joined these aggregated fixation and saccade features into a single DataFrame called `ivt_1_features`, using `QuestionKey` as the index. We also filled any resulting missing values (from `QuestionKey` values that may only have fixations or saccades, but not both) with 0. This `ivt_1_features` DataFrame now provides a consolidated summary of key eye-tracking characteristics for each question, which can be used for further analysis or modeling.